# Cologrossi Experiment
(Run model in the shell)

## Dambreak flow - Collagrosi and Landrini (2003)


The problem consists of a 0.60m x 1.20m (height x width) column of
water, initially at rest, that collapses under the action of gravity
and impacts to a wall.  The computational domain is a rectangular box
with a height of 1.80m and a width of 3.22m.  The top of the domain is
left open, when the rest of the boundary patches act as free slip walls.
In the following figure, a sketch of the dambreak initial conditions
is shown.

![dambreakColagrossi.bmp](dambreakColagrossi.bmp)

This case tests the ability of PROTEUS to simulate the free-surface
evolution and forces / pressures on structures, according to data that
are available in the following references.  For more details, see
runfiles or references.

### References

- Colagrossi, A. & Landrini, M. (2003) Numerical simulation of
  interfacial flows by smoothed particle hydrodynamics. Journal of
  Computational Physics, 191, 448-475. [DOI: 10.1016/S0021-9991(03)00324-3](https://www.researchgate.net/publication/222543591_Numerical_Simulation_of_Interfacial_Flows_by_Smoothed_Particle_Hydrodynamics)

- Martin, J. C. & Moyce, W. J., (1952) Part IV. An Experimental Study
  of the Collapse of Liquid Columns on a Rigid Horizontal Plane.
  Phil. Trans. R. Soc. Lond. A. 244, (882), 312-324. [DOI: 10.1098/rsta.1952.0006](http://rsta.royalsocietypublishing.org/content/244/882/312)

- Zhou, Z. Q., De Kat, J. O. & Buchner, B. (1999) A nonlinear 3-D
  approach to simulate green water dynamics on deck. J. Piquet
  (Ed.), Proc. 7th Int. Conf. Num. Ship Hydrod., Nantes, 5.11, 15. [PDF](http://www.marin.nl/upload_mm/9/0/e/1803494324_1999999096_Zhou,_de_Kat_%26_Buchner_-_Numerical_Hydrodynamics_1999_-_3D_approach_green_seas.pdf)


In [ ]:
!parun dambreak_Colagrossi_so.py -l 5 -O ../../../inputTemplates/petsc.options.superlu_dist

TriangleBaseMesh nbase=1 baseFlags= VApq30Dena0.00032142 
ApplyTriangulate flags= VApq30Dena0.00032142
Constructing Delaunay triangulation by divide-and-conquer method.
  Sorting vertices.
  Forming triangulation.
  Removing ghost triangles.
Delaunay milliseconds:  0
Recovering segments in Delaunay triangulation.
    Constructing mapping from vertices to triangles.
  Recovering PSLG segments.
Segment milliseconds:  0
Removing unwanted triangles.
  Marking concavities (external triangles) for elimination.
Spreading regional attributes.
Hole milliseconds:  0
Adding Steiner points to enforce quality.
  Looking for encroached subsegments.
  Making a list of bad triangles.
  Splitting bad triangles.
Quality milliseconds:  32

Writing vertices.
Writing triangles.
Writing segments.
Writing edges.
Writing neighbors.

Output milliseconds:  6
Total running milliseconds:  39

Statistics:

  Input vertices: 4
  Input segments: 4
  Input holes: 0

  Mesh vertices: 14469
  Mesh triangles: 28466
  Me

In [1]:
from tables import  openFile
archive = openFile('dambreak_Colagrossi_p.h5','r')
import dambreak_Colagrossi
import dambreak_Colagrossi_so
import matplotlib.tri as mtri
from matplotlib import pyplot as  plt
import numpy as np
domain = dambreak_Colagrossi.domain
nodes = archive.getNode("/nodesSpatial_Domain0")
x=nodes[:,0]
y=nodes[:,1]
elements = archive.getNode("/elementsSpatial_Domain0")
triang = mtri.Triangulation(x, y, elements)
xg = np.linspace(0, domain.L[0], 20)
yg = np.linspace(0, domain.L[1], 20)
xi, yi = np.meshgrid(xg,yg)
plt.figure()
for it,t in enumerate(dambreak_Colagrossi_so.tnList[:]):
    phi = archive.getNode("/phi_t"+`it`)
    vof = archive.getNode("/vof_t"+`it`)
    wvof = np.ones(vof.shape,'d')
    wvof -= vof
    u = archive.getNode("/u_t"+`it`)
    v = archive.getNode("/v_t"+`it`)
    plt.clf()
    plt.xlabel(r'z[m]')
    plt.ylabel(r'x[m]')
    colors = ['b','g','r','c','m','y','k','w']
    plt.xlim(domain.x[0]-0.1*domain.L[0],domain.x[0]+domain.L[0]+0.1*domain.L[0])    
    for si,s in enumerate(domain.segments):
        plt.plot([domain.vertices[s[0]][0],
                     domain.vertices[s[1]][0]],
                    [domain.vertices[s[0]][1],
                     domain.vertices[s[1]][1]],
                    color=colors[domain.segmentFlags[si]-1],
                    linewidth=2,
                    marker='o')
    plt.tricontourf(x,y,elements,wvof*np.sqrt(u[:]**2 + v[:]**2))
    plt.tricontour(x,y,elements,phi,[0], linewidth=4)
    u_interp_lin = mtri.LinearTriInterpolator(triang, u[:])
    v_interp_lin = mtri.LinearTriInterpolator(triang, v[:])
    u_lin = u_interp_lin(xi, yi)
    v_lin = v_interp_lin(xi, yi)
    plt.streamplot(xg, yg, u_lin, v_lin,color='k')
    plt.title('T=%2.2f' % (t,))
    plt.axis('equal')
    plt.xlim((0,domain.L[0]))
    plt.savefig('phi%4.4d.png' % (it,))

In [2]:
Image('phi0001.png')

NameError: name 'Image' is not defined

In [3]:
!rm -f dambreakColagrossi.mp4; avconv -i phi%4d.png -vcodec libx264 dambreakColagrossi.mp4

avconv: /home/jovyan/proteus/linux2/lib/liblzma.so.5: no version information available (required by /lib/x86_64-linux-gnu/libsystemd.so.0)
avconv version 11.8-6:11.8-1~deb8u1, Copyright (c) 2000-2016 the Libav developers
  built on Oct  1 2016 07:16:29 with gcc 4.9.2 (Debian 4.9.2-10)
Input #0, image2, from 'phi%4d.png':
  Duration: 00:00:12.08, start: 0.000000, bitrate: N/A
    Stream #0.0: Video: png, bgra, 432x288, 25 fps, 25 tbn
[libx264 @ 0xfa8040] using cpu capabilities: MMX2 SSE2Fast LZCNT
[libx264 @ 0xfa8040] profile High, level 2.1
[libx264 @ 0xfa8040] 264 - core 142 r2431 a5831aa - H.264/MPEG-4 AVC codec - Copyleft 2003-2014 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=-2 threads=3 lookahead_threads=1 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bfram

In [4]:
from IPython.core.display import HTML
data_uri_mp4 = open("dambreakColagrossi.mp4", "rb").read().encode("base64").replace("\n", "")
video_tag = """<video controls>
<source type ="video/mp4" src="data:video/mp4;base64,{mp4}"/>
Your browser does not support the video tag
</video>""".format(mp4=data_uri_mp4)
HTML(data=video_tag)